# TP11 - tas de sable - sujet

<span class='alert-info'> Répondez aux questions dans un notebook séparé que vous rendrez au format <strong> NOM1_NOM2_tp11.ipynb </strong>. </span>

Dans ce TP, nous allons modéliser des tas de sable. 

## 1. Programmation orientée objet 

Pour commencer, je vous suggère de jeter un coup d'œil à la façon dont [les concepts objet sont implémentés en Python](https://gayerie.dev/docs/python/python3/objet.html). Un exemple est présenté dans le notebook joint **exemple_POO.ipynb**.  

## 2. Tas de sable 

Les tas de sable seront pour nous (on pourrait considérer des versions *plus générales*) des grilles carrées $n \times n$ dont chaque case peut contenir un certain nombre (entier naturel) de **grains de sable**. L'état d'un tel *tas de sable* sera représenté par une matrice dont les entrées correspondent au nombre de grains sur chaque case.

Le fichier ```Sandpile.sage``` contient un squelette de classe pour modéliser un tas de sable. Vous pouvez éditer ce fichier dans votre éditeur de texte préféré. Pour charger le fichier, on utilise la commande ```load```. 

In [1]:
load("Sandpile.sage")

-= The sandpile class is loaded! =-

Don't forget to reload the class each time the file is modified!


On crée un tas de sable à partir de la matrice des grains de sable. 

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>
    
Implémenter dans ```Sandpile.sage``` une méthode ```show(self)``` renvoyant une image représentant la grille. Vous êtes libres de choisir la façon de représenter les nombres de grains par des couleurs, mais vous devez pouvoir gérer des tailles de grilles et nombres de grains arbitrairement grands. Par exemple, avec le tas de sable ```s``` instancié à partir de la matrice: 
$$
\begin{bmatrix}
1 & 2 & 3 \\ 
2 & 3 & 4 \\ 
3 & 4 & 5 
\end{bmatrix}
$$

---

Pour la représentation graphique des tas de sable j'ai choisit de représenter chaque case par un cube de côté 1 et de hauteur égale au nombre de grains de sable qu'elle contient.

```python
  def show(self):
    field = [] # liste des cubes
    for i in range(self.mat.nrows()):
      for j in range(self.mat.ncols()):
        # on ajoute un cube de hauteur égale au nombre de grains de sable et de couleur proportionnelle à la hauteur à une coordonnée égale à la coordonnée du cube dans la grille
        field.append(Box([1/2,1/2, self.mat[i,j]/2], color=(1, 1 - self.mat[i,j]/10, 0.2)).translate([i+1/2,j+1/2,self.mat[i,j]/2]))
    show(sum(field), shade="low") # on affiche la somme des cubes pour avoir une représentation 3D
```

In [2]:
# create the s object (call the __init__ method)
s = Sandpile([[1,2,3], [2,3,4], [3,4,5]])

# print the s object (call the __repr__ method)
print(s)

# show the s object (call the show method)
s.show()

[1 2 3]
[2 3 4]
[3 4 5]


Graphics3d Object

## 3. Tas instables

Les cases contenant quatre grains de sable ou plus sont **instables** et peuvent se renverser (*topple*). Lors du renversement, quatre grains sont prélevés de la case et se répartissent chez les quatre voisins immédiats de celle-ci ; si des grains tombent hors de la grille, ils sont perdus.

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

Implémenter une méthode ```topple(self,i,j)``` qui fait se renverser la case $(i,j)$ de ```self``` si elle est instable. Si la case $(i,j)$ n'est pas instable, rien ne se passe. On convient de numéroter les cases de $(0,0)$ en haut à gauche à $(n-1,n-1)$ en bas à droite.

Par exemple avec le tas ```s``` précédent:

- d'abord ```s.topple(1,2)``` devrait mettre ```s == Sandpile([[1,2,4],[2,4,0],[3,4,6]])``` à ```True```
- puis ```s.topple(1,1)``` devrait mettre ```s == Sandpile([[1,3,4], [3,0,1], [3,5,6]])``` à ```True```

---

Pour implémenter cette méthode, j'ai utilisé la méthode ```topple(self, i, j)``` qui renvoie ```True``` si la case $(i,j)$ a été renversée et ```False``` sinon. Si la case $(i,j)$ a été renversée, alors les cases voisines sont modifiées en conséquence.

```python
  def topple(self, i, j) -> bool:
    if self.mat[i,j] >= 4: # on vérifie si la case est instable
      self.mat[i,j] -= 4 # on retire 4 grains de sable à la case
      # on ajoute 1 grain de sable aux cases voisines si elles existent
      if i > 0:
        self.mat[i-1,j] += 1
      if i < self.mat.nrows()-1:
        self.mat[i+1,j] += 1
      if j > 0:
        self.mat[i,j-1] += 1
      if j < self.mat.ncols()-1:
        self.mat[i,j+1] += 1
      # on renvoie True si la case a été renversée (ça va servir pour la méthode suivante)
      return True
    return False
```

In [3]:
s.topple(1, 2)
print(s == Sandpile([[1,2,4],[2,4,0],[3,4,6]]))

s.topple(1, 1)
print(s == Sandpile([[1,3,4], [3,0,1], [3,5,6]]))

s.show()

True
True


Graphics3d Object

## 4. Stabilisation d'un tas

À force de laisser se renverser les cases instables, on finit par arriver à un tas dans lequel **toutes** les cases sont stables (s'il y a trop de grains sur la grille, ils finiront par tomber en dehors). Le résultat fondament de la théorie est que le tas stable ainsi obtenu **ne dépend pas de l'ordre des renversements effectués**.

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

- Implémenter la méthode `stabilize(self)` qui renverse les cases instables une à une tant qu'il en reste. On comptera le nombre de renversements effectués pour atteindre une configuration stable. On testera que le tas ```Sandpile([[1,3,4], [3,0,1], [3,5,6]])``` se stabilise en 6 renversements vers ```Sandpile([[3,0,1],[0,3,3],[1,3,3]])```. 

- Observer la stabilisation du tas de sable $21 \times 21$ dont chaque case contient 2 grains, sauf celle du milieu qui en contient 100. 

---

Pour implémenter cette méthode, j'ai utilisé une boucle ```while``` qui s'arrête lorsque toutes les cases sont stables. Pour chaque case, on vérifie si elle est instable et si c'est le cas, on la renverse. On compte le nombre de renversements effectués pour atteindre une configuration stable.

```python
  def stabilize(self) -> int:
    """Stabilize the sandpile.

    Returns:
        int: The number of iterations needed to stabilize the sandpile.
    """
    count = 0 # variable qui compte le nombre de renversements
    while True:
      for i in range(self.mat.nrows()):
        for j in range(self.mat.ncols()):
          count += 1 if self.topple(i,j) else 0 # on incrémente count si la case a été renversée
      if max([self.mat[i,j] for i in range(self.mat.nrows()) for j in range(self.mat.ncols())]) < 4: # on vérifie si toutes les cases sont stables
        break # on arrête la boucle si c'est le cas
    return count
```

Explication ```if max([self.mat[i,j] for i in range(self.mat.nrows()) for j in range(self.mat.ncols())]) < 4:``` : on vérifie si toutes les cases sont stables en regardant si la case la plus haute contient moins de 4 grains de sable. On utilise une liste de compréhension pour parcourir toutes les cases de la grille.

In [4]:
s = Sandpile([[1,3,4], [3,0,1], [3,5,6]])
s.show() # unstable
count = s.stabilize()
s.show() # stable
print(f"La pile est stable : {s == Sandpile([[3,0,1],[0,3,3],[1,3,3]])}, en {count} itérations.")

Graphics3d Object

Graphics3d Object

La pile est stable : True, en 6 itérations.


In [5]:
s = Sandpile([[2 for i in range(21)] for j in range(21)]) # On crée une pile de sable de taille 21x21 remplie de 2 grains de sable
s.mat[10,10] = 100 # On ajoute 100 grains de sable au milieu de la pile
count = s.stabilize() # On stabilise la pile
s.show()
print(f"La pile s'est stabilisé en {count} itérations.")

Graphics3d Object

La pile s'est stabilisé en 1522 itérations.


La stabilisation ne se fait pas efficacement, car après avoir check si une case est instable, et que aucune case ne c'est renversée à coté, on check toutes les cases à nouveau.
Dans notre cas du 21x21, toute les cases exterieures non même pas eu la possibilité d'être instable car aucune case n'a pu se renverser à coté d'elles.
Une solution pour améliorer l'algorithme serait de check que les cases voisines à une case qui vient de se renverser.

## 5. Loi de composition interne

Soient deux tas de sables $s$ et $t$. On définit le tas de sable $s + t$ en: 
- ajoutant case à case tous les grains du tas de sable $t$ au tas de sable $s$ 
- en stabilisant le tas obtenu. 

L'ensemble des tas stables $n \times n$ muni de $+$ a une structure de monoïde commutatif, le neutre étant le tas vide.

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

Définir l'opérateur d'addition sur les tas de sable. On notera que ```a+b``` en Python est équivalent à ```a.__add__(b)```. On vérifiera que les tas de sables instanciés sur les matrices: 

$$
\begin{bmatrix}
1 & 1 & 1 & 0 & 1 \\ 
1 & 0 & 3 & 1 & 3 \\ 
0 & 2 & 2 & 2 & 1 \\ 
0 & 1 & 2 & 1 & 0 \\ 
3 & 0 & 0 & 1 & 0 
\end{bmatrix} \qquad \text{ et } \qquad 
\begin{bmatrix}
1 & 0 & 2 & 0 & 3 \\ 
0 & 2 & 2 & 1 & 0 \\ 
3 & 1 & 2 & 1 & 1 \\ 
0 & 0 & 3 & 3 & 3 \\ 
3 & 1 & 3 & 1 & 3 
\end{bmatrix}
$$

donnent, après addition, le tas de sable instancié par la matrice:

$$
\begin{bmatrix}
2 & 3 & 1 & 3 & 1 \\ 
3 & 2 & 3 & 3 & 2 \\ 
1 & 1 & 1 & 2 & 2 \\ 
3 & 3 & 1 & 3 & 3 \\ 
3 & 1 & 0 & 3 & 1 
\end{bmatrix}
$$


---

Grace à la surcharge d'opérateur, on peux avoir une syntaxe plus naturelle pour l'addition de deux tas de sable avec la fonctionnalité que l'on veut.

Dans notre cas, on veut que l'addition de deux tas de sable renvoie un nouveau tas de sable qui est la somme des deux tas de sable, et qui est stabilisé.

```python
  def __add__(self, other) -> Sandpile:
    s_tmp = self.mat + other.mat # on additionne les deux matrices, qui elle aussi on été surchargé
    s_tmp = Sandpile(s_tmp) # on instancie un nouveau tas de sable avec la matrice obtenue
    s_tmp.stabilize() # on stabilise le tas de sable
    return s_tmp
```

In [6]:
s1 = Sandpile([[1,1,1,0,1],[1,0,3,1,3],[0,2,2,2,1],[0,1,2,1,0],[3,0,0,1,0]])
s2 = Sandpile([[1,0,2,0,3],[0,2,2,1,0],[3,1,2,1,1],[0,0,3,3,3],[3,1,3,1,3]])

s = s1 + s2
print(s)
s.show()

assert_s = Sandpile([[2,3,1,3,1],[3,2,3,3,2],[1,1,1,2,2],[3,3,1,3,3],[3,1,0,3,1]])
print(f"Notre pile est-elle égale à la pile attendue ? {s == assert_s}")

[2 3 1 3 1]
[3 2 3 3 2]
[1 1 1 2 2]
[3 3 1 3 3]
[3 1 0 3 1]


Graphics3d Object

Notre pile est-elle égale à la pile attendue ? True


## 6. Tas récurrents

Parmi les tas stables, celui dont chaque case contient 3 grains de sable est particulier: il est impossible d'y ajouter quoi que ce soit (sauf le tas vide) **sans créer d'éboulis**. 

Les tas stables obtenus en ajoutant des grains à ce tas stable maximal sont appelés *tas récurrents* car ils ont tendance à apparaître beaucoup plus souvent que ceux qui ne le sont pas. 

**En symboles** : les tas récurrents sont les tas de la forme $s_{\text{max}} + t$, où $s_{\text{max}} $ désigne le tas de sable maximal (trois grains de sable sur chaque case) et $t$ n'importe quel tas.

<div class='alert-info'>  <strong> Théorème </strong> : L'ensemble des tas récurrents forme un monoïde commutatif pour la loi +. </div>

Il n'est pas évident de dire qui est le neutre de ce monoïde. En admettant qu'il existe, on peut le déterminer de la façon suivante :

- le potentiel neutre $s_0$ s'écrit $s_0 = s_{\text{max}}  + t$ où $t$ est à déterminer. 

- le tas maximal $s_{\text{max}}$ est un tas récurrent (car $s_{\text{max}} = s_{\text{max}}  + 0$) donc avec la définition du neutre: 

$$
s_{\text{max}}  + s_0 = s_{\text{max}}  \qquad \text{ soit } \qquad s_{\text{max}}  + (s_{\text{max}}  + t) = s_{\text{max}} 
$$

- cela permet de déterminer $t$ : c'est le tas qu'il faut ajouter à $s_{\text{max}}  + s_{\text{max}}$ pour remplir chaque case à capacité maximale. Autrement dit, $t$ est le complément à $s_{\text{max}}$ du tas stabilisé $s_{\text{max}}  + s_{\text{max}}$. 

On peut vérifier, une fois cette description obtenue, que le tas $s_0 = s_{\text{max}}  + t$ défini ainsi est bien un neutre pour l'ensemble des tas récurrents.

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

- Calculer et vérifier que le neutre $s_0$ pour les tas récurrents $5 \times 5$ est le tas de sable correspondant à la matrice: 

$$
\begin{bmatrix}
2 & 3 & 2 & 3 & 2 \\ 
3 & 2 & 1 & 2 & 3 \\ 
2 & 1 & 0 & 1 & 2 \\ 
3 & 2 & 1 & 2 & 3 \\ 
2 & 3 & 2 & 3 & 2 
\end{bmatrix}
$$

- Calculer et afficher le neutre pour les tas récurrents $10 \times 10$. 

---

Pour s'aider à repondre au question j'ai introduit deux nouvelles méthodes statique dans la classe Sandpile:

```python
  @staticmethod
  def get_random(n, max=10):
    return Sandpile([[randint(0,max) for i in range(n)] for j in range(n)])
  
  @staticmethod
  def get_max(n):
    return Sandpile( n*[ n*[3] ] )
```

Elles permettent de générer aléatoirement un tas de sable de taille $n \times n$ et de générer le tas de sable maximal de taille $n \times n$.

Si s0 est le neutre pour les tas récurrents, alors si on ajoute s0 à un tas récurrent, on obtient ce même tas récurrent.

In [7]:
smax = Sandpile.get_max(5)
srand = Sandpile.get_random(5)
sreq = smax + srand # On créé notre tas récurrent

sneutre = Sandpile([[2,3,2,3,2],[3,2,1,2,3],[2,1,0,1,2],[3,2,1,2,3],[2,3,2,3,2]])

s = sreq + sneutre # On ajoute le tas neutre à notre tas récurrent
print(f"Nos deux piles sont elle les même ? {s == sreq}")

Nos deux piles sont elle les même ? True


Donc, la matrice donnée donne bien un tas de sable neutre.

On introduit une nouvelle méthode statique qui permet de calculer le neutre pour les tas récurrents de taille $n \times n$.

```python
  @staticmethod
  def find_neutral(n: int) -> Sandpile:
    smax = Sandpile.get_max(n) # on récupère le tas de sable maximal
    smax2 = smax + smax
    t = Sandpile([[3 - smax2.mat[i,j] for i in range(n)] for j in range(n)]) # on calcule le complément à smax de smax2
    sneutre = smax + t # on calcule le neutre
    return sneutre
```

In [8]:

sn = Sandpile.find_neutral(10)
sn.show()
sreq = Sandpile.get_random(10) + Sandpile.get_max(10)
print(f"A t'on trouvé un tas neutre ? {sreq + sn == sreq}")

Graphics3d Object

A t'on trouvé un tas neutre ? True


Avec toute c'est méthodes sur notre classe de tas de sable, on peux plus facilement utiliser notre nouvelle structure de façon plus naturelle. C'est comme ça que beaucoup de structure mathématique sont implémenté dans sagemath, par exemple la classe `Matrix` qui permet de manipuler des matrices. Ca permet au utilisateur de ne pas avoir à se soucier de la façon dont les opérations sont implémenté, mais de pouvoir les utiliser comme si c'était des opérations de base.